In [1]:
import warnings 
warnings.filterwarnings('ignore')

# lire films

In [2]:
import pandas as pd 
movies = pd.read_csv('ml-latest/movies.csv', header=0)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# nombre des films

In [3]:
movies.shape

(9742, 3)

# lire les notes

In [4]:
ratings = pd.read_csv('ml-latest/ratings.csv', header=0)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# nombre des notes

In [5]:
ratings.shape

(100836, 4)

# faire test et former dataa a travers annee de film par example 20% data test and 80% former data

# vérifier si il y a un column vide dans films

In [6]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

# fait le test sur tout qui plus grand que la date 2015

# diviser la date par le titre

In [7]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')
movies['year'] = movies['year'].fillna('1999')

movies.head() 

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


# divisier data a test apres 2015 et former avant 2015

In [8]:
threshold_date = '2015'
ids = movies[movies['year'] < threshold_date]['movieId'].values

training_data = ratings[ratings['movieId'].isin(ids)]
print(f'Training data size :{training_data.shape}')
testing_data = ratings[~ratings['movieId'].isin(ids)]
print(f'Testing data size :{testing_data.shape}')

Training data size :(98411, 4)
Testing data size :(2425, 4)


In [9]:
def get_favorite_movies(user_id, ratings_df):
    favorites = ratings_df[(ratings_df['userId'] == user_id) & (ratings_df['rating'] >= 3.5)].sort_values(by='rating', ascending=False)['movieId']
    return set(favorites if type(favorites) == pd.Series else [favorites])

# Recall et Precision 
recall = recommandé et il aime / nombre de recommende
precision = recommandé et il aime / tous que recommandé qu'il aime

mais il y a un pproblem 
quand il donne 1000 recommende par example il fais seulement sur un nombre aleatoire 

et on fais une chose c'est precision@k et recall@k 
par example:
5 recommendation 10111 
et prescision@k k=3 il vas prend 2 qui recommende et les recommendation qu'il aime par user sur nombre k  donc 2/3 
et recall@k k=3 il vas prend 2 qui recommende et les recommendation qu'il aime par user sur nombre de tous les recommendations qu'il aime 4 donc 2/4

In [10]:
class ModelEvaluator:
    def __init__(self, training_data, testing_data, threshold=3.5):
        self.training_data = training_data
        self.testing_data = testing_data
        self.threshold = threshold 
    
    def evaluate_model_for_user(self, model, user_id):
        # Obtenir les films favoris dans le test
        favorites_in_test = get_favorite_movies(user_id, self.testing_data)
        
        # Obtenir les recommandations du modèle pour l'utilisateur donné
        # à travers ses films favoris, en utilisant les données d'entraînement
        person_recs_df = model.recommend_items(user_id, items_to_ignore=get_favorite_movies(user_id, self.training_data))
        
        # Filtrer les recommandations en fonction du seuil et trier par note prédite
        person_recs_df = person_recs_df[person_recs_df['predicted_rating'] >= self.threshold].sort_values(by='predicted_rating', ascending=False)
        
        # Calculer le nombre de films réellement recommandés qui sont des favoris dans le test
        true_relevant = person_recs_df[person_recs_df['movieId'].isin(favorites_in_test)].shape[0]
        
        # Sélectionner les 5 premières recommandations
        top_5_recommended = person_recs_df.head(5)
        
        # Sélectionner les 10 premières recommandations
        top_10_recommended = person_recs_df.head(10)
        
        # Calculer les nombres de hits@5 et hits@10
        hits_at_5_count = top_5_recommended[top_5_recommended['movieId'].isin(favorites_in_test)].shape[0]
        hits_at_10_count = top_10_recommended[top_10_recommended['movieId'].isin(favorites_in_test)].shape[0]
        
        # Calculer la précision@5 et le rappel@5
        precision_at_5 = hits_at_5_count / top_5_recommended.shape[0] if top_5_recommended.shape[0] != 0 else 1
        recall_at_5 = hits_at_5_count / true_relevant if true_relevant != 0 else 1
        
        # Calculer la précision@10 et le rappel@10
        precision_at_10 = hits_at_10_count / top_10_recommended.shape[0] if top_10_recommended.shape[0] != 0 else 1
        recall_at_10 = hits_at_10_count / true_relevant if true_relevant != 0 else 1
        
        person_metrics = {
            'hits@5_count': hits_at_5_count,
            'hits@10_count': hits_at_10_count,
            'recommended@5_count': top_5_recommended.shape[0],
            'recommended@10_count': top_10_recommended.shape[0],
            'relevents': true_relevant,
            'recall@5': recall_at_5,
            'recall@10': recall_at_10,
            'precision@5': precision_at_5,
            'precision@10': precision_at_10
        }
        return person_metrics
    
    def evaluate_model(self, model):
        users_metrics = []
        users_ids = list(set(self.testing_data['userId'].values))
        for idx, user_id in enumerate(users_ids):
            metrics = self.evaluate_model_for_user(model, user_id)
            metrics['user_id'] = user_id
            users_metrics.append(metrics)
        print('%d users processed' % idx)
        
        detailed_results_df = pd.DataFrame(users_metrics).sort_values('hits@5_count', ascending=False)

        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['relevents'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['relevents'].sum())
        
        global_precision_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['recommended@5_count'].sum())
        global_precision_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['recommended@10_count'].sum())
        
        global_metrics = {
            'modelName': model.get_model_name(),
            'recall@5': global_recall_at_5,
            'recall@10': global_recall_at_10,
            'precision@5': global_precision_at_5,
            'precision@10': global_precision_at_10
        }
        return global_metrics, detailed_results_df

model_evaluator = ModelEvaluator(training_data, testing_data)


# Popularity Recommender  
la moyenne des notes par rapport les nombre des utilisateurs qui fais les notes
(v/(v+m)*r)+(v/(v+m)*c)
v nombre des notes dans le film 
m note minimale requise pour être répertorié
r la moyenne des notes
c la moyenne des notes sur l'ensemble des films

In [11]:
popularity = ratings.groupby('movieId').agg({'rating':['mean', 'count']}).reset_index()
popularity.columns = ['movieId','ratings_mean', 'ratings_count']
popularity.sort_values(by='ratings_mean', ascending=False)

,movieId,ratings_mean,ratings_count
7638,88448,5.0,1
8089,100556,5.0,1
9065,143031,5.0,1
9076,143511,5.0,1
9078,143559,5.0,1
...,...,...,...
9253,157172,0.5,1
7536,85334,0.5,1
6486,53453,0.5,1
5200,8494,0.5,1


In [12]:
class PopularityRecommender:
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularities_df):
        self.popularities_df = popularities_df
    
    def get_model_name(self):
        return self.MODEL_NAME
    
    def weighted_rating(self, x, m, C):
        V = x['ratings_count']
        R = x['ratings_mean']
        return (V/(V+m)*R) + (m/(m+V)* C)
                
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        C = self.popularities_df['ratings_mean'].mean()
        
        self.popularities_df['predicted_rating'] = self.popularities_df.apply(lambda x: self.weighted_rating(x,3.5,C), axis=1) 
        
        recommendations_df = self.popularities_df[~self.popularities_df['movieId'].isin(items_to_ignore)].sort_values('predicted_rating', ascending=False).head(topn)
        return recommendations_df
    
popularity_model = PopularityRecommender(popularity)
        

In [13]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\n Global metrics: \n%s'%pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
124 users processed

 Global metrics: 
{'modelName': 'Popularity', 'recall@5': 0.8888888888888888, 'recall@10': 1.0, 'precision@5': 0.0128, 'precision@10': 0.0072}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,user_id
16,1,1,5,10,1,1.0,1.0,0.2,0.1,25
70,1,1,5,10,1,1.0,1.0,0.2,0.1,212
94,1,1,5,10,1,1.0,1.0,0.2,0.1,338
11,1,1,5,10,1,1.0,1.0,0.2,0.1,18
72,1,1,5,10,1,1.0,1.0,0.2,0.1,233
89,1,1,5,10,1,1.0,1.0,0.2,0.1,318
109,1,2,5,10,2,0.5,1.0,0.2,0.2,414
68,1,1,5,10,1,1.0,1.0,0.2,0.1,209
90,0,0,5,10,0,1.0,1.0,0.0,0.0,319
81,0,0,5,10,0,1.0,1.0,0.0,0.0,279


In [14]:
popularity_model.recommend_items(5)

,movieId,ratings_mean,ratings_count,predicted_rating
277,318,4.429022,317,4.416283
9600,177593,4.750000,8,4.297267
840,1104,4.475000,20,4.294407
659,858,4.289062,192,4.270683
796,1041,4.590909,11,4.270246
2224,2959,4.272936,218,4.256969
882,1178,4.541667,12,4.252811
2579,3451,4.545455,11,4.235763
921,1221,4.259690,129,4.233348
602,750,4.268041,97,4.233021


In [15]:
popularity_model.recommend_items(10)

,movieId,ratings_mean,ratings_count,predicted_rating
277,318,4.429022,317,4.416283
9600,177593,4.750000,8,4.297267
840,1104,4.475000,20,4.294407
659,858,4.289062,192,4.270683
796,1041,4.590909,11,4.270246
2224,2959,4.272936,218,4.256969
882,1178,4.541667,12,4.252811
2579,3451,4.545455,11,4.235763
921,1221,4.259690,129,4.233348
602,750,4.268041,97,4.233021


In [16]:
ids = popularity_model.recommend_items(10)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[318, 'Shawshank Redemption, The (1994)', 'Crime|Drama', '1994'],
       [750,
        'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
        'Comedy|War', '1964'],
       [858, 'Godfather, The (1972)', 'Crime|Drama', '1972'],
       [1041, 'Secrets & Lies (1996)', 'Drama', '1996'],
       [1104, 'Streetcar Named Desire, A (1951)', 'Drama', '1951'],
       [1178, 'Paths of Glory (1957)', 'Drama|War', '1957'],
       [1221, 'Godfather: Part II, The (1974)', 'Crime|Drama', '1974'],
       [2959, 'Fight Club (1999)', 'Action|Crime|Drama|Thriller', '1999'],
       [3451, "Guess Who's Coming to Dinner (1967)", 'Drama', '1967'],
       [177593, 'Three Billboards Outside Ebbing, Missouri (2017)',
        'Crime|Drama', '2017']], dtype=object)

# Content-Based Filtering model 


# utiliser genres comme content

In [17]:
#get all unique genres
genres = list(set([x for genres in movies['genres'].values for x in genres.split('|')]))
genres

['Action',
 'Drama',
 'Animation',
 'Thriller',
 'War',
 'Film-Noir',
 'Horror',
 'Fantasy',
 'Western',
 'Crime',
 'Musical',
 'Sci-Fi',
 'IMAX',
 'Comedy',
 'Children',
 '(no genres listed)',
 'Documentary',
 'Mystery',
 'Romance',
 'Adventure']

In [18]:
expanded_movies_df = movies.copy()
for g in genres:
    expanded_movies_df[g] = [0 if g in genres.split('|') else 1 for genres in movies['genres'].values]
expanded_movies_df.head()
    

,movieId,title,genres,year,Action,Drama,Animation,Thriller,War,Film-Noir,...,Musical,Sci-Fi,IMAX,Comedy,Children,(no genres listed),Documentary,Mystery,Romance,Adventure
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,1,1,0,1,1,1,...,1,1,1,0,0,1,1,1,1,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,0,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,1,0,1,1,1,1,...,1,1,1,0,1,1,1,1,0,1
4,5,Father of the Bride Part II (1995),Comedy,1995,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def normalize(value, old_max, old_min,new_max=5.0, new_min=0.0):
    old_range = (old_max - old_min)
    new_range = (new_max - new_min)
    return (((value - old_min) * new_range) / old_range) + new_min


In [20]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self, items_df, training_data, testing_data):
        self.items_df = items_df
        self.features_names = items_df.columns[4:]
        self.training_data = training_data
        self.testing_data = testing_data
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_item_profile(self, item_id):
        #take vector
        item_profile = self.items_df[self.items_df['movieId'] == item_id][self.features_names]
        return item_profile
    def get_items_profiles(self, ids):
        item_profiles = self.items_df[self.items_df['movieId'].isin(ids)][self.features_names].values
        return item_profiles
    def build_users_profile(self, user_id):
        user_df = self.training_data[self.training_data['userId'] == user_id]
        user_items_profiles = self.get_items_profiles(user_df['movieId'].values)
        
        user_items_ratings = np.array(user_df['rating'].values).reshape(-1,1)
        user_profile = np.sum(np.multiply(user_items_profiles, user_items_ratings), axis=0) / np.sum(user_items_ratings)
        return user_profile
    
    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        user_profile = self.build_users_profile(user_id).reshape(1, -1)
        cosine_similarities = cosine_similarity(user_profile,self.items_df[self.features_names].values)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_items = sorted([(self.items_df.loc[i, 'movieId'], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'predicted_rating']).head(topn)
        recommendations_df['predicted_rating'] = recommendations_df['predicted_rating'].apply(lambda x : normalize(x, 1.0, 0.0))
        return recommendations_df
content_based_recommender_model = ContentBasedRecommender(expanded_movies_df, training_data, testing_data)

In [21]:
print('Evaluating Content Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content Based Filtering model...
124 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 1.0, 'recall@10': 1.0, 'precision@5': 0.0032, 'precision@10': 0.0016}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,user_id
53,2,2,5,10,2,1.0,1.0,0.4,0.2,111
0,0,0,5,10,0,1.0,1.0,0.0,0.0,2
78,0,0,5,10,0,1.0,1.0,0.0,0.0,256
91,0,0,5,10,0,1.0,1.0,0.0,0.0,326
90,0,0,5,10,0,1.0,1.0,0.0,0.0,319
89,0,0,5,10,0,1.0,1.0,0.0,0.0,318
88,0,0,5,10,0,1.0,1.0,0.0,0.0,306
87,0,0,5,10,0,1.0,1.0,0.0,0.0,305
86,0,0,5,10,0,1.0,1.0,0.0,0.0,298
85,0,0,5,10,0,1.0,1.0,0.0,0.0,296


In [22]:
tags = pd.read_csv('ml-latest/tags.csv',header=0)
tags.head(10)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [23]:
def check_name(x):
    words = x.split()
    return all(x[0].isupper() and x[1:].islower() for x in words if len(x) > 1)

def clean(x):
    if x.istitle() or check_name(x):
        return x.replace(" ", "").lower()
    else:
        return x.lower().strip()


In [24]:
tags['tag'] = tags['tag'].apply(lambda x: clean(x))
tags.head(10)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,boxing story,1445715207
4,2,89774,mma,1445715200
5,2,89774,tomhardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,leonardo dicaprio,1445715051
8,2,106782,martinscorsese,1445715056
9,7,48516,way too long,1169687325


In [25]:
expended_movies_df = movies.copy()
movies_ids = expended_movies_df['movieId'].values.tolist()
expended_movies_df['soup'] = [" ".join(tags[tags['movieId'] == id]['tag'].values.tolist()) for id in movies_ids]


In [26]:
expended_movies_df.head(10)

,movieId,title,genres,year,soup
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,fantasy magic board game robinwilliams game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,
4,5,Father of the Bride Part II (1995),Comedy,1995,pregnancy remake
5,6,Heat (1995),Action|Crime|Thriller,1995,
6,7,Sabrina (1995),Comedy|Romance,1995,remake
7,8,Tom and Huck (1995),Adventure|Children,1995,
8,9,Sudden Death (1995),Action,1995,
9,10,GoldenEye (1995),Action|Adventure|Thriller,1995,


In [27]:
expended_movies_df.loc[8, 'soup']

''

In [28]:
expended_movies_df[expended_movies_df['soup'] != ''].shape

(1572, 5)

In [29]:
def fill_empty_tag(x):
    if x['soup'] == '' :
        return " ".join(x['genres'].lower().split('|'))
    return x['soup']

expended_movies_df['soup'] = expended_movies_df.apply(lambda x: fill_empty_tag(x), axis=1)
expended_movies_df.head(10)

,movieId,title,genres,year,soup
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,fantasy magic board game robinwilliams game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,comedy drama romance
4,5,Father of the Bride Part II (1995),Comedy,1995,pregnancy remake
5,6,Heat (1995),Action|Crime|Thriller,1995,action crime thriller
6,7,Sabrina (1995),Comedy|Romance,1995,remake
7,8,Tom and Huck (1995),Adventure|Children,1995,adventure children
8,9,Sudden Death (1995),Action,1995,action
9,10,GoldenEye (1995),Action|Adventure|Thriller,1995,action adventure thriller


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(expended_movies_df['soup'])
count_matrix.shape

(9742, 1586)

In [31]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self, items_ids, items_matrix, training_data, testing_data):
        self.items_ids = items_ids
        self.items_matrix = items_matrix
        self.training_data = training_data
        self.testing_data = testing_data
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_item_profile(self, item_id):
        idx = self.items_ids.index(item_id)
        return self.items_matrix[idx].toarray().reshape(-1)
    
    def get_items_profiles(self, ids):
        item_profiles = np.array([self.get_item_profile(x) for x in ids])
        return item_profiles
    def build_users_profile(self, user_id):
        user_df = self.training_data[self.training_data['userId'] == user_id]
        user_items_profiles = self.get_items_profiles(user_df['movieId'].values)
        
        user_items_ratings = np.array(user_df['rating'].values).reshape(-1,1)
        user_profile = np.sum(np.multiply(user_items_profiles, user_items_ratings), axis=0) / np.sum(user_items_ratings)
        return user_profile
    
    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        user_profile = self.build_users_profile(user_id).reshape(1, -1)
        cosine_similarities = cosine_similarity(user_profile,self.items_matrix.toarray())
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_items = sorted([(self.items_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'predicted_rating']).head(topn)
        recommendations_df['predicted_rating'] = recommendations_df['predicted_rating'].apply(lambda x : normalize(x, 1.0, 0.0))
        return recommendations_df
content_based_recommender_model = ContentBasedRecommender(expanded_movies_df['movieId'].values.tolist(), count_matrix, training_data, testing_data)

In [32]:
print('Evaluating Content Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content Based Filtering model...
124 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.4444444444444444, 'recall@10': 1.0, 'precision@5': 0.011560693641618497, 'precision@10': 0.01355421686746988}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,user_id
100,1,1,5,10,1,1.0,1.0,0.2,0.100000,365
44,1,1,5,10,1,1.0,1.0,0.2,0.100000,596
51,1,1,5,6,1,1.0,1.0,0.2,0.166667,103
119,1,1,5,10,1,1.0,1.0,0.2,0.100000,475
85,0,0,0,0,0,1.0,1.0,1.0,1.000000,296
81,0,0,0,0,0,1.0,1.0,1.0,1.000000,279
82,0,0,5,6,0,1.0,1.0,0.0,0.000000,282
83,0,0,5,6,0,1.0,1.0,0.0,0.000000,291
84,0,0,5,10,0,1.0,1.0,0.0,0.000000,292
0,0,0,0,0,0,1.0,1.0,1.0,1.000000,2


In [33]:
content_based_recommender_model.recommend_items(8)

,movieId,predicted_rating
0,296,4.281834
1,1732,2.066995
2,2959,1.944809
3,81132,1.890180
4,20,1.850736
5,5027,1.850736
6,5628,1.850736
7,1432,1.850736
8,7007,1.850736
9,145,1.850736


In [34]:
ids = content_based_recommender_model.recommend_items(8)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[20, 'Money Train (1995)', 'Action|Comedy|Crime|Drama|Thriller',
        '1995'],
       [145, 'Bad Boys (1995)', 'Action|Comedy|Crime|Drama|Thriller',
        '1995'],
       [296, 'Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller',
        '1994'],
       [1432, 'Metro (1997)', 'Action|Comedy|Crime|Drama|Thriller',
        '1997'],
       [1732, 'Big Lebowski, The (1998)', 'Comedy|Crime', '1998'],
       [2959, 'Fight Club (1999)', 'Action|Crime|Drama|Thriller', '1999'],
       [5027, 'Another 48 Hrs. (1990)',
        'Action|Comedy|Crime|Drama|Thriller', '1990'],
       [5628, 'Wasabi (2001)', 'Action|Comedy|Crime|Drama|Thriller',
        '2001'],
       [7007, 'Last Boy Scout, The (1991)',
        'Action|Comedy|Crime|Drama|Thriller', '1991'],
       [81132, 'Rubber (2010)',
        'Action|Adventure|Comedy|Crime|Drama|Film-Noir|Horror|Mystery|Thriller|Western',
        '2010']], dtype=object)

In [35]:
ids = content_based_recommender_model.recommend_items(60)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[1015, 'Homeward Bound: The Incredible Journey (1993)',
        'Adventure|Children|Drama', '1993'],
       [1551, 'Buddy (1997)', 'Adventure|Children|Drama', '1997'],
       [1907, 'Mulan (1998)',
        'Adventure|Animation|Children|Comedy|Drama|Musical|Romance',
        '1998'],
       [2950, 'Blue Lagoon, The (1980)', 'Adventure|Drama|Romance',
        '1980'],
       [4964, 'Baran (2001)', 'Adventure|Drama|Romance', '2001'],
       [8534, 'Two Brothers (Deux frères) (2004)',
        'Adventure|Children|Drama', '2004'],
       [42418, 'New World, The (2005)', 'Adventure|Drama|Romance',
        '2005'],
       [56152, 'Enchanted (2007)',
        'Adventure|Animation|Children|Comedy|Fantasy|Musical|Romance',
        '2007'],
       [74282,
        'Anne of Green Gables: The Sequel (a.k.a. Anne of Avonlea) (1987)',
        'Children|Drama|Romance', '1987'],
       [108540, 'Ernest & Célestine (Ernest et Célestine) (2012)',
        'Adventure|Animation|Children|Comedy|Drama|Rom

In [36]:
ids = content_based_recommender_model.recommend_items(60)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[1015, 'Homeward Bound: The Incredible Journey (1993)',
        'Adventure|Children|Drama', '1993'],
       [1551, 'Buddy (1997)', 'Adventure|Children|Drama', '1997'],
       [1907, 'Mulan (1998)',
        'Adventure|Animation|Children|Comedy|Drama|Musical|Romance',
        '1998'],
       [2950, 'Blue Lagoon, The (1980)', 'Adventure|Drama|Romance',
        '1980'],
       [4964, 'Baran (2001)', 'Adventure|Drama|Romance', '2001'],
       [8534, 'Two Brothers (Deux frères) (2004)',
        'Adventure|Children|Drama', '2004'],
       [42418, 'New World, The (2005)', 'Adventure|Drama|Romance',
        '2005'],
       [56152, 'Enchanted (2007)',
        'Adventure|Animation|Children|Comedy|Fantasy|Musical|Romance',
        '2007'],
       [74282,
        'Anne of Green Gables: The Sequel (a.k.a. Anne of Avonlea) (1987)',
        'Children|Drama|Romance', '1987'],
       [108540, 'Ernest & Célestine (Ernest et Célestine) (2012)',
        'Adventure|Animation|Children|Comedy|Drama|Rom

# Collaborative filtering

In [37]:
users_items_pivot_matrix_df = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)
users_items_pivot_matrix_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_items_pivot_matrix

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [39]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [40]:
U.shape

(610, 15)

In [41]:
Vt.shape

(15, 9724)

In [42]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [43]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[ 2.48313562e+00,  1.35885909e+00,  1.10637440e+00, ...,
        -3.49319264e-03, -3.49319264e-03, -2.47602496e-02],
       [ 4.22544294e-02,  1.03805400e-02,  3.20579362e-02, ...,
         9.95659214e-03,  9.95659214e-03,  1.31613972e-02],
       [ 1.38016774e-02,  2.86395855e-02,  3.19043736e-02, ...,
         9.42462913e-04,  9.42462913e-04, -1.75137511e-03],
       ...,
       [ 2.17103278e+00,  1.92389820e+00,  1.71015130e+00, ...,
        -5.55603096e-02, -5.55603096e-02, -9.01278061e-03],
       [ 8.38486877e-01,  6.37344297e-01,  3.19950208e-01, ...,
         4.27295325e-04,  4.27295325e-04,  1.65256080e-03],
       [ 5.86830717e+00, -1.00804430e-01, -1.49624909e+00, ...,
         2.66883644e-02,  2.66883644e-02,  8.02635842e-02]])

In [44]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns=users_items_pivot_matrix_df.columns, index = users_items_pivot_matrix_df.index)

In [45]:
preds_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,2.483136,1.358859,1.106374,-0.025920,0.174511,1.614364,0.167845,0.043874,0.206104,1.823636,...,-0.003493,-0.002994,-0.003992,-0.003992,-0.003493,-0.003992,-0.003493,-0.003493,-0.003493,-0.024760
2,0.042254,0.010381,0.032058,0.002357,0.030592,-0.001568,0.013115,0.000889,0.019802,-0.139452,...,0.009957,0.008534,0.011379,0.011379,0.009957,0.011379,0.009957,0.009957,0.009957,0.013161
3,0.013802,0.028640,0.031904,-0.002689,-0.022141,0.064227,0.000262,0.009224,0.012006,0.038832,...,0.000942,0.000808,0.001077,0.001077,0.000942,0.001077,0.000942,0.000942,0.000942,-0.001751
4,2.082146,0.162685,0.102233,0.053761,0.146481,0.454800,0.304220,-0.061288,-0.030823,-0.003292,...,-0.003688,-0.003162,-0.004215,-0.004215,-0.003688,-0.004215,-0.003688,-0.003688,-0.003688,-0.014209
5,1.383537,0.957738,0.406646,0.129537,0.536569,0.756295,0.610276,0.130696,0.103396,1.163210,...,-0.000702,-0.000602,-0.000802,-0.000802,-0.000702,-0.000802,-0.000702,-0.000702,-0.000702,0.000739
6,3.083983,3.400135,1.883538,0.405848,1.867406,2.189975,2.306211,0.605855,0.471779,3.916900,...,0.005509,0.004722,0.006296,0.006296,0.005509,0.006296,0.005509,0.005509,0.005509,-0.011051
7,1.995476,0.789781,-0.082301,-0.031872,0.097296,0.860021,0.001919,-0.084484,-0.036732,1.176269,...,-0.010775,-0.009236,-0.012314,-0.012314,-0.010775,-0.012314,-0.010775,-0.010775,-0.010775,0.005820
8,1.672878,1.220942,0.575672,0.163430,0.705200,1.024762,0.788211,0.164804,0.153066,1.603970,...,-0.001049,-0.000899,-0.001199,-0.001199,-0.001049,-0.001199,-0.001049,-0.001049,-0.001049,0.002347
9,0.366844,0.042292,-0.025592,-0.011815,-0.004028,-0.002142,-0.044961,-0.022251,-0.020231,0.078649,...,-0.000115,-0.000099,-0.000131,-0.000131,-0.000115,-0.000131,-0.000115,-0.000115,-0.000115,0.002419


In [46]:
preds_df = preds_df.apply(lambda x: normalize(x, all_user_predicted_ratings.max(), all_user_predicted_ratings.min()))

In [47]:
preds_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,2.609299,2.202104,2.110658,1.700559,1.773152,2.294644,1.770737,1.725837,1.784594,2.370439,...,1.708681,1.708862,1.708501,1.708501,1.708681,1.708501,1.708681,1.708681,1.708681,1.700979
2,1.725250,1.713706,1.721557,1.710800,1.721026,1.709379,1.714697,1.710269,1.717119,1.659439,...,1.713553,1.713037,1.714068,1.714068,1.713553,1.714068,1.713553,1.713553,1.713553,1.714713
3,1.714945,1.720319,1.721502,1.708972,1.701928,1.733209,1.710041,1.713287,1.714295,1.724011,...,1.710288,1.710239,1.710337,1.710337,1.710288,1.710337,1.710288,1.710288,1.710288,1.709312
4,2.464067,1.768868,1.746974,1.729418,1.762999,1.874668,1.820130,1.687749,1.698783,1.708754,...,1.708611,1.708801,1.708420,1.708420,1.708611,1.708420,1.708611,1.708611,1.708611,1.704800
5,2.211042,2.056824,1.857227,1.756863,1.904283,1.983865,1.930979,1.757283,1.747395,2.131243,...,1.709692,1.709729,1.709656,1.709656,1.709692,1.709656,1.709692,1.709692,1.709692,1.710214
6,2.826916,2.941421,2.392134,1.856938,2.386292,2.503121,2.545220,1.929377,1.880817,3.128586,...,1.711942,1.711657,1.712227,1.712227,1.711942,1.712227,1.711942,1.711942,1.711942,1.705944
7,2.432676,1.995993,1.680139,1.698403,1.745186,2.021433,1.710641,1.679348,1.696643,2.135973,...,1.706044,1.706602,1.705487,1.705487,1.706044,1.705487,1.706044,1.706044,1.706044,1.712054
8,2.315837,2.152152,1.918446,1.769138,1.965359,2.081099,1.995424,1.769636,1.765385,2.290879,...,1.709567,1.709621,1.709512,1.709512,1.709567,1.709512,1.709567,1.709567,1.709567,1.710797
9,1.842812,1.725264,1.700677,1.705667,1.708488,1.709171,1.693663,1.701888,1.702619,1.738432,...,1.709905,1.709911,1.709899,1.709899,1.709905,1.709899,1.709905,1.709905,1.709905,1.710823


In [48]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df):
        self.predictions_df = predictions_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        sorted_user_predictions = self.predictions_df.loc[user_id].sort_values(ascending=False)
        recommendations = {'movieId': sorted_user_predictions.index, 'predicted_rating':sorted_user_predictions.values}
        recommendations_df = pd.DataFrame(recommendations)
        recommendations_df = recommendations_df[~recommendations_df['movieId'].isin(items_to_ignore)].sort_values('predicted_rating', ascending = False).head(topn)
        return recommendations_df
cf_recommender_model = CFRecommender(preds_df)

In [49]:
model_evaluator = ModelEvaluator(training_data, testing_data, 2.5)

print("Evaluating collaborative Filtering (SVD MATRIX FACTORIZATION) model ...")
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\n Global metrics: \n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating collaborative Filtering (SVD MATRIX FACTORIZATION) model ...
124 users processed

 Global metrics: 
{'modelName': 'Collaborative Filtering', 'recall@5': 0.46511627906976744, 'recall@10': 1.0, 'precision@5': 0.06622516556291391, 'precision@10': 0.07948243992606285}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,user_id
109,5,9,5,10,9,0.555556,1.0,1.0,0.900000,414
50,5,8,5,10,8,0.625000,1.0,1.0,0.800000,610
76,5,9,5,10,9,0.555556,1.0,1.0,0.900000,249
89,2,3,5,10,3,0.666667,1.0,0.4,0.300000,318
57,1,3,5,10,3,0.333333,1.0,0.2,0.300000,125
42,1,1,5,7,1,1.000000,1.0,0.2,0.142857,73
34,1,2,5,10,2,0.500000,1.0,0.2,0.200000,62
71,0,0,2,2,0,1.000000,1.0,0.0,0.000000,227
82,0,0,5,10,0,1.000000,1.0,0.0,0.000000,282
91,0,0,4,4,0,1.000000,1.0,0.0,0.000000,326


In [50]:
cf_recommender_model.recommend_items(5)

,movieId,predicted_rating
0,318,2.655146
1,356,2.651065
2,296,2.601698
3,150,2.595818
4,457,2.552295
5,110,2.546781
6,593,2.511089
7,590,2.505336
8,480,2.459162
9,380,2.423270


In [51]:
cf_recommender_model.recommend_items(6)

,movieId,predicted_rating
0,457,4.214001
1,110,4.180563
2,150,4.171748
3,356,4.095240
4,590,4.080451
5,380,4.023972
6,318,3.891560
7,296,3.883220
8,364,3.864499
9,592,3.850389


In [52]:
ids = cf_recommender_model.recommend_items(8)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[110, 'Braveheart (1995)', 'Action|Drama|War', '1995'],
       [150, 'Apollo 13 (1995)', 'Adventure|Drama|IMAX', '1995'],
       [296, 'Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller',
        '1994'],
       [318, 'Shawshank Redemption, The (1994)', 'Crime|Drama', '1994'],
       [356, 'Forrest Gump (1994)', 'Comedy|Drama|Romance|War', '1994'],
       [380, 'True Lies (1994)',
        'Action|Adventure|Comedy|Romance|Thriller', '1994'],
       [457, 'Fugitive, The (1993)', 'Thriller', '1993'],
       [480, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi|Thriller',
        '1993'],
       [590, 'Dances with Wolves (1990)', 'Adventure|Drama|Western',
        '1990'],
       [593, 'Silence of the Lambs, The (1991)', 'Crime|Horror|Thriller',
        '1991']], dtype=object)

In [53]:

ids = content_based_recommender_model.recommend_items(8)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[20, 'Money Train (1995)', 'Action|Comedy|Crime|Drama|Thriller',
        '1995'],
       [145, 'Bad Boys (1995)', 'Action|Comedy|Crime|Drama|Thriller',
        '1995'],
       [296, 'Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller',
        '1994'],
       [1432, 'Metro (1997)', 'Action|Comedy|Crime|Drama|Thriller',
        '1997'],
       [1732, 'Big Lebowski, The (1998)', 'Comedy|Crime', '1998'],
       [2959, 'Fight Club (1999)', 'Action|Crime|Drama|Thriller', '1999'],
       [5027, 'Another 48 Hrs. (1990)',
        'Action|Comedy|Crime|Drama|Thriller', '1990'],
       [5628, 'Wasabi (2001)', 'Action|Comedy|Crime|Drama|Thriller',
        '2001'],
       [7007, 'Last Boy Scout, The (1991)',
        'Action|Comedy|Crime|Drama|Thriller', '1991'],
       [81132, 'Rubber (2010)',
        'Action|Adventure|Comedy|Crime|Drama|Film-Noir|Horror|Mystery|Thriller|Western',
        '2010']], dtype=object)

In [83]:
class HybridRecommender:
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cf_model, cb_model, expended_movies_df):
        self.cf_model = cf_model
        self.cb_model = cb_model
        self.expended_movies_df = expended_movies_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        cf_recommendations = self.cf_model.recommend_items(user_id, items_to_ignore=items_to_ignore, topn=topn)
        cb_recommendations = self.cb_model.recommend_items(user_id, items_to_ignore=items_to_ignore, topn=topn)
        hybrid_recommendations = pd.concat([cf_recommendations, cb_recommendations]).drop_duplicates().reset_index(drop=True)
        return hybrid_recommendations.sample(frac=1).head(topn)
    
    def recommend_items_by_title(self, title, topn=10):
        # Find the movie based on the provided title
        movie = self.expended_movies_df[self.expended_movies_df['title'].str.contains(title, case=False)]

        if movie.empty:
            return pd.DataFrame(columns=['movieId'])

        # Extract the genres from the found movie
        query_genres = movie.iloc[0]['genres']

        # Initialize list to store similar movie ids
        similar_movies = []

        # Find other movies with similar genres
        for index, row in self.expended_movies_df.iterrows():
            if row['title'] != movie.iloc[0]['title'] and row['genres'] == query_genres:
                similar_movies.append({'movieId': row['movieId']})

        if similar_movies:
            return pd.DataFrame(similar_movies).head(topn)
        else:
            return pd.DataFrame(columns=['movieId'])


# Create instance of hybrid recommender
hybrid_model = HybridRecommender(cf_model=cf_recommender_model, cb_model=content_based_recommender_model, expended_movies_df=expended_movies_df)


# Evaluate hybrid recommender
print("Evaluating Hybrid recommendation model ...")
hybrid_global_metrics, hybrid_detailed_results_df = model_evaluator.evaluate_model(hybrid_model)
print('\n Global metrics: \n%s' % hybrid_global_metrics)
hybrid_detailed_results_df.head(10)


Evaluating Hybrid recommendation model ...
124 users processed

 Global metrics: 
{'modelName': 'Hybrid', 'recall@5': 0.40625, 'recall@10': 1.0, 'precision@5': 0.024667931688804556, 'precision@10': 0.03975155279503106}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,user_id
50,3,6,5,10,6,0.500000,1.0,0.60,0.600000,610
34,1,2,5,10,2,0.500000,1.0,0.20,0.200000,62
100,1,1,5,6,1,1.000000,1.0,0.20,0.166667,365
106,1,1,5,6,1,1.000000,1.0,0.20,0.166667,401
107,1,1,5,5,1,1.000000,1.0,0.20,0.200000,408
44,1,2,5,10,2,0.500000,1.0,0.20,0.200000,596
77,1,1,4,4,1,1.000000,1.0,0.25,0.250000,252
76,1,6,5,10,6,0.166667,1.0,0.20,0.600000,249
42,1,1,5,8,1,1.000000,1.0,0.20,0.125000,73
59,1,1,5,10,1,1.000000,1.0,0.20,0.100000,141


In [55]:
ids = hybrid_model.recommend_items(8)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[296, 'Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller',
        '1994'],
       [318, 'Shawshank Redemption, The (1994)', 'Crime|Drama', '1994'],
       [356, 'Forrest Gump (1994)', 'Comedy|Drama|Romance|War', '1994'],
       [480, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi|Thriller',
        '1993'],
       [590, 'Dances with Wolves (1990)', 'Adventure|Drama|Western',
        '1990'],
       [593, 'Silence of the Lambs, The (1991)', 'Crime|Horror|Thriller',
        '1991'],
       [1432, 'Metro (1997)', 'Action|Comedy|Crime|Drama|Thriller',
        '1997'],
       [5027, 'Another 48 Hrs. (1990)',
        'Action|Comedy|Crime|Drama|Thriller', '1990'],
       [5628, 'Wasabi (2001)', 'Action|Comedy|Crime|Drama|Thriller',
        '2001'],
       [7007, 'Last Boy Scout, The (1991)',
        'Action|Comedy|Crime|Drama|Thriller', '1991']], dtype=object)

In [56]:
ids = content_based_recommender_model.recommend_items(8)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[20, 'Money Train (1995)', 'Action|Comedy|Crime|Drama|Thriller',
        '1995'],
       [145, 'Bad Boys (1995)', 'Action|Comedy|Crime|Drama|Thriller',
        '1995'],
       [296, 'Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller',
        '1994'],
       [1432, 'Metro (1997)', 'Action|Comedy|Crime|Drama|Thriller',
        '1997'],
       [1732, 'Big Lebowski, The (1998)', 'Comedy|Crime', '1998'],
       [2959, 'Fight Club (1999)', 'Action|Crime|Drama|Thriller', '1999'],
       [5027, 'Another 48 Hrs. (1990)',
        'Action|Comedy|Crime|Drama|Thriller', '1990'],
       [5628, 'Wasabi (2001)', 'Action|Comedy|Crime|Drama|Thriller',
        '2001'],
       [7007, 'Last Boy Scout, The (1991)',
        'Action|Comedy|Crime|Drama|Thriller', '1991'],
       [81132, 'Rubber (2010)',
        'Action|Adventure|Comedy|Crime|Drama|Film-Noir|Horror|Mystery|Thriller|Western',
        '2010']], dtype=object)

In [57]:
ids = cf_recommender_model.recommend_items(8)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[110, 'Braveheart (1995)', 'Action|Drama|War', '1995'],
       [150, 'Apollo 13 (1995)', 'Adventure|Drama|IMAX', '1995'],
       [296, 'Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller',
        '1994'],
       [318, 'Shawshank Redemption, The (1994)', 'Crime|Drama', '1994'],
       [356, 'Forrest Gump (1994)', 'Comedy|Drama|Romance|War', '1994'],
       [380, 'True Lies (1994)',
        'Action|Adventure|Comedy|Romance|Thriller', '1994'],
       [457, 'Fugitive, The (1993)', 'Thriller', '1993'],
       [480, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi|Thriller',
        '1993'],
       [590, 'Dances with Wolves (1990)', 'Adventure|Drama|Western',
        '1990'],
       [593, 'Silence of the Lambs, The (1991)', 'Crime|Horror|Thriller',
        '1991']], dtype=object)

In [74]:
ids =hybrid_model.recommend_items_by_title("Godfather, The")['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[16, 'Casino (1995)', 'Crime|Drama', '1995'],
       [1213, 'Goodfellas (1990)', 'Crime|Drama', '1990'],
       [1245, "Miller's Crossing (1990)",
        'Crime|Drama|Film-Noir|Thriller', '1990'],
       [1466, 'Donnie Brasco (1997)', 'Crime|Drama', '1997'],
       [2247, 'Married to the Mob (1988)', 'Comedy', '1988'],
       [2249, 'My Blue Heaven (1990)', 'Comedy', '1990']], dtype=object)

In [86]:
ids =hybrid_model.recommend_items_by_title("Taxi Driver")['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[149, 'Amateur (1994)', 'Crime|Drama|Thriller', '1994'],
       [259, 'Kiss of Death (1995)', 'Crime|Drama|Thriller', '1995'],
       [456, 'Fresh (1994)', 'Crime|Drama|Thriller', '1994'],
       [482, 'Killing Zoe (1994)', 'Crime|Drama|Thriller', '1994'],
       [507, 'Perfect World, A (1993)', 'Crime|Drama|Thriller', '1993'],
       [707, 'Mulholland Falls (1996)', 'Crime|Drama|Thriller', '1996'],
       [1344, 'Cape Fear (1962)', 'Crime|Drama|Thriller', '1962'],
       [1351, 'Blood and Wine (Blood & Wine) (1996)',
        'Crime|Drama|Thriller', '1996'],
       [1598, 'Desperate Measures (1998)', 'Crime|Drama|Thriller',
        '1998'],
       [1647, 'Playing God (1997)', 'Crime|Drama|Thriller', '1997']],
      dtype=object)